In [1]:
import pandas as pd
import numpy as np
import sqlite3
import regex

In [2]:
source = '/Users/patrickokwir/Desktop/Git_Projects/Ted-Talks-Recommender-System/Data_output/ted_talk_clean.csv'
talks = pd.read_csv(source, index_col=0)
talks.head()

,author,talk,description,likes,views,transcript
0,Machine Dazzle,how to unleash your inner maximalist through c...,tapping into the transformational power of cos...,8100,270192,"Hello, I am Machine Dazzle, and I am an emotio..."
2,Jioji Ravulo,a liberating vision of identity that transcend...,how can we move past societys inclination to b...,9200,309952,Can you paint with all the colors of the wind?...
3,Rebecca Darwent,how to fund real change in your community,is there a way to give back that benefits ever...,1000,341218,I spent my whole career in the nonprofit secto...
4,Susanne Buckley-Zistel,what caused the rwandan genocide,for one hundred days in 1994 the african count...,3700,126376,"For 100 days in 1994, the African country of R..."
5,Conor Russomanno,a powerful new neurotech tool for augmenting y...,in an astonishing talk and tech demo neurotech...,1100,374259,I became obsessed with the relationship betwee...


In [3]:
# create sqlite3 connection
conn = sqlite3.connect('/Users/patrickokwir/Desktop/Git_Projects/Ted-Talks-Recommender-System/SRC/talks.db')
c = conn.cursor()

# fetch all talks
c.execute('SELECT * FROM topics2')
topics = c.fetchall()

# create dataframe

In [4]:
#topics to dataframe
topics = pd.DataFrame(topics, columns=['id', 'title', 'author', 'date', 'topic', 'url'], index=None)
topics.head()


,id,title,author,date,topic,url
0,803,\n\nA modern argument for the rights of animal...,Peter Singer,\nJun 2023\n,activism,ted.com/talks/peter_singer_a_modern_argument_f...
1,804,"\n\nWhy Iranians are cutting their hair for ""W...",Sahar Zand,\nJun 2023\n,activism,ted.com/talks/sahar_zand_why_iranians_are_cutt...
2,805,"\n\n""Woman, Life, Freedom"" in Iran — and what ...",Golshifteh Farahani,\nMay 2023\n,activism,ted.com/talks/golshifteh_farahani_woman_life_f...
3,806,\n\nPussy Riot's powerful message to Vladimir ...,Nadya Tolokonnikova,\nApr 2023\n,activism,ted.com/talks/nadya_tolokonnikova_pussy_riot_s...
4,807,\n\nHow one small idea led to $1 million of pa...,Tiffani Ashley Bell,\nMar 2023\n,activism,ted.com/talks/tiffani_ashley_bell_how_one_smal...


In [5]:
# remove '\n\n' from title column
topics['title'] = topics['title'].apply(lambda x: x.replace('\n\n', ''))
topics['date'] = topics['date'].apply(lambda x: x.replace('\n', ''))

In [6]:
# group by title and author
topics = topics.groupby(['title', 'author', 'date'])['topic'].apply(', '.join).reset_index()

In [7]:
# reorganize columns —> title, author, date, topic, url
topics = topics[['title', 'author', 'date', 'topic']]

In [8]:
topics['tags'] = topics['topic'].apply(lambda x: x.replace('+', '_'))

In [19]:
topics = topics[['title', 'author', 'date', 'tags']]
# lowercase title
topics['title'] = topics['title'].apply(lambda x: x.lower())
# remove punctuation from title
topics['title'] = topics['title'].apply(lambda x: regex.sub(r'[^\w\s]', '', x))

topics['title'] = topics['title'].apply(lambda x: x.strip())
topics['title'] = topics['title'].apply(lambda x: x.replace('[^\w\s]', ''))

In [10]:
topics.shape

(5993, 4)

In [11]:
# count duplicates
topics.duplicated().sum()

0

In [12]:
# count null values
topics.isnull().sum()

title     0
author    0
date      0
tags      0
dtype: int64

In [13]:
# rename column 'talks' to 'title'
talks.rename(columns={'talk': 'title'}, inplace=True)

In [20]:
# merge talks and topics on author and title
df = pd.merge(talks, topics, on=['title', 'author'], how='left')
df.head()

,author,title,description,likes,views,transcript,date,tags
0,Machine Dazzle,how to unleash your inner maximalist through c...,tapping into the transformational power of cos...,8100,270192,"Hello, I am Machine Dazzle, and I am an emotio...",Jun 2023,"art, creativity, design, fashion, performance"
1,Jioji Ravulo,a liberating vision of identity that transcend...,how can we move past societys inclination to b...,9200,309952,Can you paint with all the colors of the wind?...,Jun 2023,"diversity, identity, inclusion, indigenous_peo..."
2,Rebecca Darwent,how to fund real change in your community,is there a way to give back that benefits ever...,1000,341218,I spent my whole career in the nonprofit secto...,Jun 2023,"business, community, equality, humanity, money..."
3,Susanne Buckley-Zistel,what caused the rwandan genocide,for one hundred days in 1994 the african count...,3700,126376,"For 100 days in 1994, the African country of R...",Jun 2023,"africa, animation, education, history, identit..."
4,Conor Russomanno,a powerful new neurotech tool for augmenting y...,in an astonishing talk and tech demo neurotech...,1100,374259,I became obsessed with the relationship betwee...,Jun 2023,"biotech, brain, disability, health, invention,..."


In [21]:
# nan values
df.isnull().sum()

author           0
title            0
description      0
likes            0
views            0
transcript       0
date           151
tags           151
dtype: int64

In [22]:
# return rows with null values
df[df.isnull().any(axis=1)]

,author,title,description,likes,views,transcript,date,tags
25,Hanako Sawada,test yourself,in 1960 composer john cage went on television ...,8800,294676,"In 1960, American composer John Cage went on ...",NaN,NaN
41,David Biello,1816,in 1815 mount tambora erupted and its emission...,8300,276992,"It’s April 10th, 1815, and in just a few mome...",NaN,NaN
80,TED-Ed,the most important century in human history,is it possible that this century is the most i...,9600,321885,What's the most important century in human his...,NaN,NaN
99,TED-Ed,what happens if an engineered virus escapes th...,since the 1970s researchers have engineered su...,1600,555843,"In the spring of 1979, a lab worker in Sverdlo...",NaN,NaN
113,TED-Ed,what if you experienced every human life in hi...,imagine that your life began as one of the pla...,5400,1807252,"Imagine that your life began roughly 300,000 ...",NaN,NaN
...,...,...,...,...,...,...,...,...
5251,Robert Fischell,my wish,accepting his 2005 ted prize inventor robert f...,1600,554937,I'm going to discuss with you three of my inve...,NaN,NaN
5252,Edward Burtynsky,my wish,accepting his 2005 ted prize photographer edwa...,4300,1440671,"Walk around for four months with three wishes,...",NaN,NaN
5279,Cameron Sinclair,my wish,accepting his 2006 ted prize cameron sinclair ...,4100,1376315,I'm going to take you on a journey very quickl...,NaN,NaN
5280,Jehane Noujaim,my wish,jehane noujaim unveils her 2006 ted prize wish...,1400,495466,I can't help but this wish: to think about whe...,NaN,NaN


In [23]:
# select rows without null values
df = df[df['tags'].notna()]

In [43]:
import sys
import os
# change directory to save file
os.chdir('/Users/patrickokwir/Desktop/Git_Projects/Ted-Talks-Recommender-System/Data_output')

In [44]:
df.to_csv('../Data_output/ted_talk_clean_merged_bert.csv')